## Ноут-шаблон под обучение моделей
> **Важно: обязательно фиксируем рандом (`seed=42`) для всех библиотек и методов, где под капотом подразумевается рандом!** 

> **Почему это важно:** для гарантии консистентности данных при сравнении разных моделей и решений.


In [27]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import random
import ast

# фиксирую то, что импортнул
random.seed(42)
np.random.seed(42)

In [28]:
# Скачиваем необходимые ресурсы (выполнить один раз)
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

### Импорт датасета и сплит на выборки

In [2]:
# Загружаем датасет
df = pd.read_csv("customer_support_dataset_generated.csv")
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


### Предобработка

In [4]:
# Обязательный блок: исправляем дублящиеся категории и интенты
def process_category_unique(value):
    if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        try:
            categories = ast.literal_eval(value)
            unique_categories = list(set(categories))
            # Если осталась одна категория - возвращаем как строку
            if len(unique_categories) == 1:
                return unique_categories[0]
            else:
                return ', '.join(sorted(unique_categories))
        except:
            return value
    else:
        return value

    
    
df['category'] = df['category'].apply(process_category_unique).astype(str)
df['intent'] = df['intent'].apply(process_category_unique).astype(str)

**Дальше я делаю подготовку текстовок обращений**

В целом, этот блок на ваше усмотрение можно кастомить, если я что-то забыл учесть, или у вас появилась классная идея, ~~или я где-то жестоко не прав и ошибся~~, но базовый минимум здесь - дропнуть английские стоп-слова и пунктуацию, привести слова к леммам

In [25]:
# Так было в моём EDA

# stop_words = set(stopwords.words('english'))  # или 'russian', 'spanish' и т.д.


# def remove_stopwords(text):
#     # Удаляем различные шаблоны с фигурными скобками
#     patterns_to_remove = [r'{{.*?}}']
    
#     for pattern in patterns_to_remove:
#         text = re.sub(pattern, '', text)
#     tokens = word_tokenize(text.lower())
#     tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
#     return " ".join(tokens)


# df['clean_instruction'] = df['instruction'].apply(remove_stopwords)

In [21]:
class TextPreprocessor:
    def __init__(self, language='english', use_lemmatization=True, custom_stopwords=None):
        
        self.stop_words = set(stopwords.words(language))
        if custom_stopwords:
            self.stop_words.update(custom_stopwords)
        
        self.lemmatizer = WordNetLemmatizer() if use_lemmatization else None
        self.stemmer = PorterStemmer() if not use_lemmatization else None
        
        # Расширенные шаблоны для удаления
        self.patterns_to_remove = [
            r'{{.*?}}',  # шаблоны с фигурными скобками
            r'\[.*?\]',  # квадратные скобки
            r'<.*?>',    # HTML теги
            r'http\S+',  # URL
            r'@\w+',     # упоминания
            r'#\w+',     # хэштеги
        ]
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        
        # Приведение к нижнему регистру
        text = text.lower()
        
        # Удаление шаблонов
        for pattern in self.patterns_to_remove:
            text = re.sub(pattern, '', text)
        
        # Удаление пунктуации и цифр
        text = text.translate(str.maketrans('', '', string.punctuation + string.digits))
        
        # Токенизация
        tokens = word_tokenize(text)
        
        # Фильтрация и нормализация
        filtered_tokens = []
        for token in tokens:
            if (len(token) > 1 and                    # убираем одиночные символы
                token.isalpha() and                   # только буквы
                token not in self.stop_words):        # не стоп-слово
                
                if self.lemmatizer:
                    token = self.lemmatizer.lemmatize(token)
                elif self.stemmer:
                    token = self.stemmer.stem(token)
                
                filtered_tokens.append(token)
        
        return " ".join(filtered_tokens)


In [30]:
# Пример использования
# Создаем препроцессор
preprocessor = TextPreprocessor(language='english', use_lemmatization=True)

df['clean_instruction'] = df['instruction'].apply(preprocessor.clean_text)

In [33]:
preprocessor.stop_words # для понимания, что там лежит 

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [31]:
df[['instruction', 'clean_instruction']].sample(25)

,instruction,clean_instruction
5994,I can't see your money back policy,cant see money back policy
5954,i want help checking in which cases can i ask ...,want help checking case ask refund
35784,What's the cost to break my agreement?,whats cost break agreement
25935,how to check the fucking rebate current status?,check fucking rebate current status
2912,I need support changing the delivery address,need support changing delivery address
19745,how can I order some products?,order product
33580,What's the customer service email?,whats customer service email
26061,I expect a restitution of {{Currency Symbol}}{...,expect restitution
15557,get bill from {{Person Name}},get bill
34858,I need to get in contact with customer support.,need get contact customer support


### Сплит выборки
Разбиваем на:
- `train` - на нём обучаем модели
- `validation` - на нём "тюним" модели, настраимаем гиперпараметры (когда GridSearch делаем, к примеру)
- `test` - финальная выборка для оценки качества  Тестовый набор используется **ТОЛЬКО ОДИН РАЗ** в самом конце! При обучении и подборе параметров модель ничего не должна знать из этого датасета.

Предлагаю бить в пропорции **70/15/15**

In [34]:
def train_val_test_split(X, 
                         y, 
                         val_size=0.15, 
                         test_size=0.15, 
                         random_state=42 # !!!
                         ):
    """
    Разделяет данные на train, validation и test наборы
    """

    # Сначала отделяем test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, 
        y, 
        test_size=test_size, 
        random_state=random_state, 
        shuffle=True
        # stratify=y  # есть классы всего лишь с 1 примером, поэтому просто шафлим
    )
    
    # Затем разделяем на train и validation
    relative_val_size = val_size / (1 - test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, 
        y_train_val, 
        test_size=relative_val_size, 
        random_state=random_state, 
        shuffle=True
        # stratify=y_train_val  # есть классы всего лишь с 1 примером, поэтому просто шафлим
    )
    
    return X_train, X_val, X_test, y_train, y_val, y_test



# Получаем выборки
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    X=df.drop(['intent'], axis=1), 
    y=df[['intent']]
)

### Векторизация
Пробуем следующие методы:
- Bag of Words (мешок слов)
- TF-IDF
- Word2Vec
- FastText

Статья, которой можно вдохновиться: https://habr.com/ru/articles/778048/

### Классификация интентов
**Модели**

Точно пробуем из линейных моделей SVM и логистическую регрессию, но в целом можно поэкспериментировать.

Для мультилейбл классификации - можно воспользоваться "One-vs-Rest" и "One-vs-One"

**Метрики:** 

f1-score, precision и recall (micro, macro и weighted - всё имплементировано в sklearn'е)